In [26]:
# https://www.kaggle.com/code/robikscube/time-series-forecasting-with-machine-learning-yt

In [27]:
# https://www.kaggle.com/competitions/godaddy-microbusiness-density-forecasting/discussion/372780

In [28]:
# https://towardsdatascience.com/interpreting-acf-and-pacf-plots-for-time-series-forecasting-af0d6db4061c
# https://www.analyticsvidhya.com/blog/2019/12/6-powerful-feature-engineering-techniques-time-series/

In [29]:
import warnings
warnings.simplefilter("ignore")

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

In [33]:
from xgboost import XGBRegressor

In [34]:
from sklearn.metrics import mean_squared_error

# 1. Load data

In [35]:
df = pd.read_csv('retail_price.csv')

In [36]:
df.sample(5)

,product_id,product_category_name,month_year,qty,total_price,freight_price,unit_price,product_name_lenght,product_description_lenght,product_photos_qty,...,comp_1,ps1,fp1,comp_2,ps2,fp2,comp_3,ps3,fp3,lag_price
305,cool2,cool_stuff,01-07-2017,16,2079.84,18.715625,129.990000,33,897,1,...,99.990000,4.2,16.529091,129.99,4.3,18.715625,129.990000,4.3,18.715625,129.990000
368,watches8,watches_gifts,01-08-2018,11,1737.40,14.592727,157.945455,35,640,1,...,112.000000,4.2,13.440000,105.00,4.2,23.385714,145.550909,3.8,21.897273,157.142857
171,garden10,garden_tools,01-06-2018,6,299.40,23.251667,49.900000,59,341,2,...,49.900000,4.1,19.890833,49.90,4.2,23.251667,49.900000,4.1,22.878667,53.709524
277,watches6,watches_gifts,01-05-2018,114,11820.20,0.095439,103.792523,33,523,3,...,103.792523,4.2,0.095439,105.00,4.2,11.377297,132.531250,3.8,15.762187,105.648625
26,garden5,garden_tools,01-01-2018,4,398.00,41.540000,99.500000,36,450,1,...,49.900000,4.1,19.495000,49.90,4.2,15.841034,49.900000,4.1,16.505128,99.000000


In [37]:
df.shape

(676, 30)

# 2. Check data quality

data type

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 676 entries, 0 to 675
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   product_id                  676 non-null    object        
 1   product_category_name       676 non-null    object        
 2   month_year                  676 non-null    datetime64[ns]
 3   qty                         676 non-null    int64         
 4   total_price                 676 non-null    float64       
 5   freight_price               676 non-null    float64       
 6   unit_price                  676 non-null    float64       
 7   product_name_lenght         676 non-null    int64         
 8   product_description_lenght  676 non-null    int64         
 9   product_photos_qty          676 non-null    int64         
 10  product_weight_g            676 non-null    int64         
 11  product_score               676 non-null    float64       
 12 

In [39]:
df['month_year'] = pd.to_datetime(df['month_year'])

In [44]:
df.rename({'month_year':'Datetime'}, axis='columns', inplace=True)

In [45]:
df

,product_id,product_category_name,Datetime,qty,total_price,freight_price,unit_price,product_name_lenght,product_description_lenght,product_photos_qty,...,comp_1,ps1,fp1,comp_2,ps2,fp2,comp_3,ps3,fp3,lag_price
0,bed1,bed_bath_table,2017-01-05,1,45.95,15.100000,45.950000,39,161,2,...,89.9,3.9,15.011897,215.000000,4.4,8.760000,45.95,4.0,15.100000,45.900000
1,bed1,bed_bath_table,2017-01-06,3,137.85,12.933333,45.950000,39,161,2,...,89.9,3.9,14.769216,209.000000,4.4,21.322000,45.95,4.0,12.933333,45.950000
2,bed1,bed_bath_table,2017-01-07,6,275.70,14.840000,45.950000,39,161,2,...,89.9,3.9,13.993833,205.000000,4.4,22.195932,45.95,4.0,14.840000,45.950000
3,bed1,bed_bath_table,2017-01-08,4,183.80,14.287500,45.950000,39,161,2,...,89.9,3.9,14.656757,199.509804,4.4,19.412885,45.95,4.0,14.287500,45.950000
4,bed1,bed_bath_table,2017-01-09,2,91.90,15.100000,45.950000,39,161,2,...,89.9,3.9,18.776522,163.398710,4.4,24.324687,45.95,4.0,15.100000,45.950000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,bed5,bed_bath_table,2017-01-05,1,215.00,8.760000,215.000000,56,162,5,...,89.9,3.9,15.011897,215.000000,4.4,8.760000,45.95,4.0,15.100000,214.950000
672,bed5,bed_bath_table,2017-01-06,10,2090.00,21.322000,209.000000,56,162,5,...,89.9,3.9,14.769216,209.000000,4.4,21.322000,45.95,4.0,12.933333,215.000000
673,bed5,bed_bath_table,2017-01-07,59,12095.00,22.195932,205.000000,56,162,5,...,89.9,3.9,13.993833,205.000000,4.4,22.195932,45.95,4.0,14.840000,209.000000
674,bed5,bed_bath_table,2017-01-08,52,10375.00,19.412885,199.509804,56,162,5,...,89.9,3.9,14.656757,199.509804,4.4,19.412885,45.95,4.0,14.287500,205.000000


In [46]:
df.sort_values('Datetime', inplace=True)

unique value

In [48]:
df.shape

(676, 30)

In [49]:
for i in df.columns:
    print('Columns name: ', i)
    print('Unique value: ', df[i].unique())
    print('Count unique value: ', df[i].nunique())
    print('-'*10)

Columns name:  product_id
Unique value:  ['health7' 'health5' 'bed2' 'garden8' 'health9' 'cool1' 'garden1'
 'computers4' 'garden3' 'watches3' 'health8' 'garden5' 'watches1' 'cool2'
 'garden9' 'garden7' 'garden2' 'cool5' 'computers1' 'garden10' 'bed1'
 'garden6' 'bed5' 'computers3' 'perfumery1' 'watches6' 'watches2'
 'watches8' 'garden4' 'furniture1' 'consoles2' 'furniture2' 'cool4'
 'consoles1' 'perfumery2' 'health2' 'watches5' 'furniture3' 'watches7'
 'health1' 'cool3' 'health4' 'computers5' 'bed4' 'bed3' 'computers2'
 'watches4' 'furniture4' 'health6' 'health3' 'computers6' 'health10']
Count unique value:  52
----------
Columns name:  product_category_name
Unique value:  ['health_beauty' 'bed_bath_table' 'garden_tools' 'cool_stuff'
 'computers_accessories' 'watches_gifts' 'perfumery' 'furniture_decor'
 'consoles_games']
Count unique value:  9
----------
Columns name:  Datetime
Unique value:  <DatetimeArray>
['2017-01-01 00:00:00', '2017-01-02 00:00:00', '2017-01-03 00:00:00',
 '2017-

missing value

In [50]:
df.isnull().sum()

product_id                    0
product_category_name         0
Datetime                      0
qty                           0
total_price                   0
freight_price                 0
unit_price                    0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_score                 0
customers                     0
weekday                       0
weekend                       0
holiday                       0
month                         0
year                          0
s                             0
volume                        0
comp_1                        0
ps1                           0
fp1                           0
comp_2                        0
ps2                           0
fp2                           0
comp_3                        0
ps3                           0
fp3                           0
lag_price                     0
dtype: int64

zero value

In [52]:
(df == 0).sum()

product_id                     0
product_category_name          0
Datetime                       0
qty                            0
total_price                    0
freight_price                  1
unit_price                     0
product_name_lenght            0
product_description_lenght     0
product_photos_qty             0
product_weight_g               0
product_score                  0
customers                      0
weekday                        0
weekend                        0
holiday                       42
month                          0
year                           0
s                              0
volume                         0
comp_1                         0
ps1                            0
fp1                            0
comp_2                         0
ps2                            0
fp2                            0
comp_3                         0
ps3                            0
fp3                            0
lag_price                      0
dtype: int

blank value

In [53]:
(df == '').sum()

product_id                    0
product_category_name         0
Datetime                      0
qty                           0
total_price                   0
freight_price                 0
unit_price                    0
product_name_lenght           0
product_description_lenght    0
product_photos_qty            0
product_weight_g              0
product_score                 0
customers                     0
weekday                       0
weekend                       0
holiday                       0
month                         0
year                          0
s                             0
volume                        0
comp_1                        0
ps1                           0
fp1                           0
comp_2                        0
ps2                           0
fp2                           0
comp_3                        0
ps3                           0
fp3                           0
lag_price                     0
dtype: int64

# 3. Exploratory data analysis (EDA)

# 3. Feature engineering

datetime features

In [ ]:
def datetime_feature_extraction(df):
    # Day extraction
    df['hour'] = df['Datetime'].dt.hour
    df['dayofweek'] = df['Datetime'].dt.dayofweek
    df['dayofmonth'] = df['Datetime'].dt.day
    df['dayofyear'] = df['Datetime'].dt.dayofyear
    df['is_weekend'] = (df['Datetime'].dt.dayofweek >= 5).astype(int)
    df['is_month_start'] = df['Datetime'].dt.is_month_start.astype(int)
    df['is_month_end'] = df['Datetime'].dt.is_month_end.astype(int)
    df['is_year_start'] = df['Datetime'].dt.is_year_start.astype(int)
    df['is_year_end'] = df['Datetime'].dt.is_year_end.astype(int)
    df['weekday_weekend'] = df['Datetime'].dt.dayofweek.apply(lambda x: 0 if x >= 5 else 1)

    # Week extraction
    df['weekofyear'] = df['Datetime'].dt.isocalendar().week
    df['weekofmonth'] = np.ceil(df['dayofmonth'] / 7).astype(int)    
    
    # Month extraction
    df['month'] = df['Datetime'].dt.month
    df['quarter'] = df['Datetime'].dt.quarter
    df['is_quarter_start'] = df['Datetime'].dt.is_quarter_start.astype(int)
    df['is_quarter_end'] = df['Datetime'].dt.is_quarter_end.astype(int)
    df['days_in_month'] = df['Datetime'].dt.days_in_month

    # Year extraction
    df['year'] = df['Datetime'].dt.year
    df['year_since_2002'] = df['Datetime'].dt.year - 2002
    df['is_leap_year'] = df['Datetime'].dt.is_leap_year.astype(int)
    df['is_odd_year'] = (df['Datetime'].dt.year % 2 != 0).astype(int) # observed from data
    return df

In [ ]:
df = datetime_feature_extraction(df)

In [ ]:
df

lag features

In [ ]:
# 1 day = 24 hrs
# 1 week = 168 hrs
# 1 month = 720 hours
# 1 quarter = 2160
# 1 year = 8760 hours

In [ ]:
def lag_feature_extraction(df):
    lag_hours = [1, 2, 3, 6, 12, 24, 168, 720, 2160, 8760]

    for lag in lag_hours:
        df[f'PJME_MW_lag_{lag}h'] = df['PJME_MW'].shift(periods=lag)
    df.fillna(method='bfill', inplace=True)

    return df

In [ ]:
df = lag_feature_extraction(df)

In [ ]:
df

rolling windows

In [ ]:
def rolling_feature_extraction(df):
    rolling_periods = [12, 24, 168, 720, 2160, 8760]

    for period in rolling_periods:
        df[f'PJME_MW_MA_{period}h_mean'] = df['PJME_MW'].rolling(window=period).mean().shift(1)
        df[f'PJME_MW_MA_{period}h_min'] = df['PJME_MW'].rolling(window=period).min().shift(1)
        df[f'PJME_MW_MA_{period}h_max'] = df['PJME_MW'].rolling(window=period).max().shift(1)
        df[f'PJME_MW_MA_{period}h_std'] = df['PJME_MW'].rolling(window=period).std().shift(1)

    return df

In [ ]:
df = rolling_feature_extraction(df)

In [ ]:
df

drop na

In [ ]:
df = df.dropna()

In [ ]:
df.isna().sum()

correlation

In [ ]:
corr = df.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 5))
sns.heatmap(corr, annot=True, ax=ax)

In [ ]:
corr_churn_table = corr['PJME_MW'].sort_values(ascending=False)

In [ ]:
corr_churn_table.head(20)

In [ ]:
corr_churn_abs_table = corr['PJME_MW'].abs().sort_values(ascending=False)

In [ ]:
corr_churn_abs_table.head(20)

# 4. Model

In [ ]:
df

In [ ]:
df_for_predict

train-test split

In [ ]:
df = df.sort_values('Datetime')

In [ ]:
df.drop(columns=['Datetime'], inplace=True)

In [ ]:
X = df.drop(columns=['PJME_MW'])
y = df['PJME_MW']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=False)

model

In [ ]:
model = XGBRegressor()

In [ ]:
tscv = TimeSeriesSplit(n_splits = 5)

In [ ]:
param_grid = {
    'n_estimators': [1000, 1500, 2000],
    'max_depth': [5, 7, 10],
    'learning_rate': [0.05, 0.1, 0.2]
}

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

evaluate

In [ ]:
y_pred = grid_search.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)

In [ ]:
-mse

# 5. Predict

In [ ]:
df_for_predict

In [ ]:
last_timestamp = df_for_predict['Datetime'].max()

In [ ]:
last_timestamp

In [ ]:
future_timestamps = [last_timestamp + pd.Timedelta(hours=i) for i in range(1, 24*365*2)]

In [ ]:
future_predictions = []

In [ ]:
# recursive

In [ ]:
for timestamp in future_timestamps:
    # Prep features
    df_feature = pd.DataFrame([[timestamp, None]], columns=['Datetime', 'PJME_MW'])
    future_predictions = pd.concat([df_for_predict, df_feature], ignore_index=True, sort=False)
    df_for_predict_feature = datetime_feature_extraction(future_predictions)
    df_for_predict_feature = lag_feature_extraction(df_for_predict_feature)
    df_for_predict_feature = rolling_feature_extraction(df_for_predict_feature)
    df_for_predict_feature = df_for_predict_feature.tail(1).iloc[:,2:]
    y_predicted = grid_search.predict(df_for_predict_feature)[0]
    # Predict
    df_predicted = pd.DataFrame([[timestamp, y_predicted]], columns=['Datetime', 'PJME_MW'])
    # Append
    df_for_predict = pd.concat([df_for_predict, df_predicted], ignore_index=True, sort=False)
    # # Print
    print(timestamp, y_predicted)

In [ ]:
df_for_predict['predict_flag'] = np.where(df_for_predict['Datetime'] <= last_timestamp, 'Actual', 'Predicted')

In [ ]:
df_for_predict

In [ ]:
plt.figure(figsize=(10, 6))
actual_data = df_for_predict[df_for_predict['predict_flag'] == 'Actual']
predicted_data = df_for_predict[df_for_predict['predict_flag'] == 'Predicted']
plt.plot(actual_data['Datetime'], actual_data['PJME_MW'], color='blue', label='Actual')
plt.plot(predicted_data['Datetime'], predicted_data['PJME_MW'], color='orange', label='Predicted')
plt.xlabel('Datetime')
plt.ylabel('PJME_MW') 
plt.title('PJME_MW over Time')
plt.legend()
plt.show()